In [1]:
import xml.etree.cElementTree as et
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.cluster import KMeans 
import xmltodict
import json
from unidecode import unidecode

In [2]:
# parse systems.xml into json
systems = []

def callback(path, item):
    systems.append(item)
    return True

with open(r'F:\systems.xml', encoding='utf8') as xmlfile:
    xmltodict.parse(xmlfile.read(), xml_attribs=False, item_depth=2, item_callback=callback, force_list=('planet'))
processed_systems = pd.DataFrame(systems)
display(processed_systems)

# parse system_connectors.xml
systems = []
with open(r'F:\system_connectors.xml', encoding='utf8') as xmlfile:
    xmltodict.parse(xmlfile.read(), xml_attribs=False, item_depth=2, item_callback=callback, force_list=('planet'))
processed_connectors = pd.DataFrame(systems)
display(processed_connectors)

processed_systems = pd.concat([processed_systems, processed_connectors])
processed_systems.rename(columns = {'xcood':'PosX', 'ycood':'PosY'}, inplace=True) # rename cols to conform to inner sphere data

display(processed_systems)

,id,xcood,ycood,spectralType,primarySlot,planet
0,A Place,-123.764,272.182,G5V,2,"[{'name': 'Sullivan', 'type': 'Giant Terrestri..."
1,Aalzorg,583.881,256.931,G9V,2,"[{'name': 'Rowan', 'type': 'Dwarf Terrestrial'..."
2,Abadan,-70.319,-95.949,G2V,5,"[{'name': 'Africus', 'type': 'Dwarf Terrestria..."
3,Abagnar,380.231,314.823,G9V,3,"[{'name': 'Bin Amr', 'type': 'Terrestrial', 'o..."
4,Abbadiyah,-317.967,246.551,G2V,2,"[{'name': 'Aswad', 'type': 'Ice Giant', 'orbit..."
...,...,...,...,...,...,...
3141,Transfer Station P9,-115.441,1037.657,G3II,2,"[{'name': 'I1', 'type': 'Terrestrial', 'orbita..."
3142,Trelleborg,109.262,920.57,A9II,2,"[{'name': 'V1', 'type': 'Dwarf Terrestrial', '..."
3143,Trinity,-248.762,761.402,G4V,3,"[{'name': 'R-1', 'type': 'Terrestrial', 'orbit..."
3144,Wark,-299.568,694.859,G0V,2,"[{'name': 'D41', 'type': 'Dwarf Terrestrial', ..."


,id,xcood,ycood,spectralType,planet
0,DPR 0101,-137,889,M7V,"[{'name': '1W1', 'type': 'Dwarf Terrestrial', ..."
1,DPR 0102,-162,894,M5V,"[{'name': 'T-1', 'type': 'Giant Terrestrial', ..."
2,DPR 0103,-187,899,M0V,"[{'name': 'FP1', 'type': 'Terrestrial', 'orbit..."
3,DPR 0104,-212,904,M1V,"[{'name': 'A6-1', 'type': 'Giant Terrestrial',..."
4,DPR 0105,-237,909,M0V,"[{'name': 'N1', 'type': 'Ice Giant', 'orbitalD..."
...,...,...,...,...,...
592,TFS 38,119,1584,G9V,"[{'name': 'H1', 'type': 'Terrestrial', 'orbita..."
593,TFS 39,113,1612,G5V,"[{'name': 'MA.1', 'type': 'Giant Terrestrial',..."
594,TFS 40,107,1640,G4V,"[{'name': 'CR1', 'type': 'Terrestrial', 'orbit..."
595,TFS 41,101,1668,M2V,"[{'name': '8V9.1', 'type': 'Giant Terrestrial'..."


,id,PosX,PosY,spectralType,primarySlot,planet
0,A Place,-123.764,272.182,G5V,2,"[{'name': 'Sullivan', 'type': 'Giant Terrestri..."
1,Aalzorg,583.881,256.931,G9V,2,"[{'name': 'Rowan', 'type': 'Dwarf Terrestrial'..."
2,Abadan,-70.319,-95.949,G2V,5,"[{'name': 'Africus', 'type': 'Dwarf Terrestria..."
3,Abagnar,380.231,314.823,G9V,3,"[{'name': 'Bin Amr', 'type': 'Terrestrial', 'o..."
4,Abbadiyah,-317.967,246.551,G2V,2,"[{'name': 'Aswad', 'type': 'Ice Giant', 'orbit..."
...,...,...,...,...,...,...
592,TFS 38,119,1584,G9V,NaN,"[{'name': 'H1', 'type': 'Terrestrial', 'orbita..."
593,TFS 39,113,1612,G5V,NaN,"[{'name': 'MA.1', 'type': 'Giant Terrestrial',..."
594,TFS 40,107,1640,G4V,NaN,"[{'name': 'CR1', 'type': 'Terrestrial', 'orbit..."
595,TFS 41,101,1668,M2V,NaN,"[{'name': '8V9.1', 'type': 'Giant Terrestrial'..."


In [3]:
# add StarSystemName column and unidecode for matching as many vanilla systems as we can
processed_systems.insert(0, 'StarSystemName', processed_systems['id'])
processed_systems['StarSystemName'] = processed_systems['StarSystemName'].apply(unidecode)

In [4]:
# absurd regex to strip roman numerals and name change dates
processed_systems['StarSystemName'] = processed_systems['StarSystemName'].str.replace('(\sI*[VX]?I*)(?!\w)(\(.*[+-]\))?', r'', regex=True)
# special cases
processed_systems['StarSystemName'] = processed_systems['StarSystemName'].str.replace('New Earth', 'Tau Ceti')
# split spectral types
processed_systems['spectralType'] = processed_systems['spectralType'].str.replace(r'(\w)(\d)(.*)', r'\1,\3,\2', regex=True)
processed_systems[['SpectralType', 'Luminosity', 'SubType']] = processed_systems['spectralType'].str.split(',', expand=True)
processed_systems.drop('spectralType', axis=1, inplace=True)
display(processed_systems)

,StarSystemName,id,PosX,PosY,primarySlot,planet,SpectralType,Luminosity,SubType
0,A Place,A Place,-123.764,272.182,2,"[{'name': 'Sullivan', 'type': 'Giant Terrestri...",G,V,5
1,Aalzorg,Aalzorg,583.881,256.931,2,"[{'name': 'Rowan', 'type': 'Dwarf Terrestrial'...",G,V,9
2,Abadan,Abadan,-70.319,-95.949,5,"[{'name': 'Africus', 'type': 'Dwarf Terrestria...",G,V,2
3,Abagnar,Abagnar,380.231,314.823,3,"[{'name': 'Bin Amr', 'type': 'Terrestrial', 'o...",G,V,9
4,Abbadiyah,Abbadiyah,-317.967,246.551,2,"[{'name': 'Aswad', 'type': 'Ice Giant', 'orbit...",G,V,2
...,...,...,...,...,...,...,...,...,...
592,TFS 38,TFS 38,119,1584,NaN,"[{'name': 'H1', 'type': 'Terrestrial', 'orbita...",G,V,9
593,TFS 39,TFS 39,113,1612,NaN,"[{'name': 'MA.1', 'type': 'Giant Terrestrial',...",G,V,5
594,TFS 40,TFS 40,107,1640,NaN,"[{'name': 'CR1', 'type': 'Terrestrial', 'orbit...",G,V,4
595,TFS 41,TFS 41,101,1668,NaN,"[{'name': '8V9.1', 'type': 'Giant Terrestrial'...",M,V,2


In [5]:
# load vanilla systems
vanilla_systems = pd.read_csv(r'F:\MW5_InnerSphereData.csv', encoding='utf16',
                          usecols=['Name', 'StarSystemName', 'PosX', 'PosY', 'SpectralType', 'Luminosity', 'SubType',
        'Cluster', 'ClusterOverlay', 'ClusterConstellation'])
display(vanilla_systems)
# merge matching systems
cols_to_use = vanilla_systems[['Name', 'StarSystemName', 'Cluster', 'ClusterOverlay', 'ClusterConstellation']]
display(cols_to_use)
processed_systems = pd.merge(processed_systems, cols_to_use, how='left', on='StarSystemName')

# clean up columns after merge
processed_systems.drop('StarSystemName', axis=1, inplace=True) # no longer need this
processed_systems.rename(columns = {'id':'StarSystemName'}, inplace=True) # rename to conform to inner sphere data

display(processed_systems)

,Name,StarSystemName,PosX,PosY,SpectralType,Luminosity,SubType,Cluster,ClusterOverlay,ClusterConstellation
0,0,INVALID,0.0,0.0,None,NaN,0,"(Id="""")",None,None
1,1,Terra,0.0,0.0,G,V,2,"(Id=""MWFactionAsset:RepairSystem_26"")",None,None
2,2,Tau Ceti,-13.0,2.0,G,V,8,"(Id=""MWFactionAsset:RepairSystem_26"")",/Game/Campaign/CampaignArcs/BorderChanges/_com...,/Game/Campaign/CampaignArcs/BorderChanges/_com...
3,3,Rigil Kentarus,-3.0,4.0,G,V,8,"(Id=""MWFactionAsset:RepairSystem_26"")",None,None
4,4,Pike,20.0,40.0,K,IV,9,"(Id="""")",None,None
...,...,...,...,...,...,...,...,...,...,...
2168,3497,HD-389202,370.0,485.0,None,NaN,0,"(Id="""")",None,None
2169,3498,HD-348126,370.0,510.0,None,NaN,0,"(Id="""")",None,None
2170,3499,HD-367482,350.0,530.0,None,NaN,0,"(Id="""")",None,None
2171,3500,HD-438297,345.0,552.0,None,NaN,0,"(Id="""")",None,None


,Name,StarSystemName,Cluster,ClusterOverlay,ClusterConstellation
0,0,INVALID,"(Id="""")",None,None
1,1,Terra,"(Id=""MWFactionAsset:RepairSystem_26"")",None,None
2,2,Tau Ceti,"(Id=""MWFactionAsset:RepairSystem_26"")",/Game/Campaign/CampaignArcs/BorderChanges/_com...,/Game/Campaign/CampaignArcs/BorderChanges/_com...
3,3,Rigil Kentarus,"(Id=""MWFactionAsset:RepairSystem_26"")",None,None
4,4,Pike,"(Id="""")",None,None
...,...,...,...,...,...
2168,3497,HD-389202,"(Id="""")",None,None
2169,3498,HD-348126,"(Id="""")",None,None
2170,3499,HD-367482,"(Id="""")",None,None
2171,3500,HD-438297,"(Id="""")",None,None


,StarSystemName,PosX,PosY,primarySlot,planet,SpectralType,Luminosity,SubType,Name,Cluster,ClusterOverlay,ClusterConstellation
0,A Place,-123.764,272.182,2,"[{'name': 'Sullivan', 'type': 'Giant Terrestri...",G,V,5,1511.0,"(Id=""MWFactionAsset:StoryCluster3"")",/Game/Campaign/CampaignArcs/Regions/StoryClust...,None
1,Aalzorg,583.881,256.931,2,"[{'name': 'Rowan', 'type': 'Dwarf Terrestrial'...",G,V,9,NaN,NaN,NaN,NaN
2,Abadan,-70.319,-95.949,5,"[{'name': 'Africus', 'type': 'Dwarf Terrestria...",G,V,2,476.0,"(Id="""")",None,None
3,Abagnar,380.231,314.823,3,"[{'name': 'Bin Amr', 'type': 'Terrestrial', 'o...",G,V,9,2048.0,"(Id="""")",None,None
4,Abbadiyah,-317.967,246.551,2,"[{'name': 'Aswad', 'type': 'Ice Giant', 'orbit...",G,V,2,1337.0,"(Id="""")",None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
3738,TFS 38,119,1584,NaN,"[{'name': 'H1', 'type': 'Terrestrial', 'orbita...",G,V,9,NaN,NaN,NaN,NaN
3739,TFS 39,113,1612,NaN,"[{'name': 'MA.1', 'type': 'Giant Terrestrial',...",G,V,5,NaN,NaN,NaN,NaN
3740,TFS 40,107,1640,NaN,"[{'name': 'CR1', 'type': 'Terrestrial', 'orbit...",G,V,4,NaN,NaN,NaN,NaN
3741,TFS 41,101,1668,NaN,"[{'name': '8V9.1', 'type': 'Giant Terrestrial'...",M,V,2,NaN,NaN,NaN,NaN


In [6]:
# add JP and HD systems
invalid_system = vanilla_systems.iloc[[0]]
#invalid_system.insert(1, 'id', invalid_system['StarSystemName']) # copy system name to id col
display(invalid_system)
extra_systems = vanilla_systems.iloc[2121:]
#extra_systems.insert(1, 'id', extra_systems['StarSystemName'])
display(extra_systems)
display(processed_systems)
processed_systems = pd.concat([processed_systems, invalid_system, extra_systems])
display(processed_systems)

,Name,StarSystemName,PosX,PosY,SpectralType,Luminosity,SubType,Cluster,ClusterOverlay,ClusterConstellation
0,0,INVALID,0.0,0.0,None,NaN,0,"(Id="""")",None,None


,Name,StarSystemName,PosX,PosY,SpectralType,Luminosity,SubType,Cluster,ClusterOverlay,ClusterConstellation
2121,3450,JP-H-405,580.0,-175.0,None,NaN,0,"(Id="""")",None,None
2122,3451,JP-W-395,595.0,-200.0,None,NaN,0,"(Id="""")",None,None
2123,3452,JP-V-405,620.0,-215.0,None,NaN,0,"(Id="""")",None,None
2124,3453,JP-S-425,645.0,-220.0,None,NaN,0,"(Id="""")",None,None
2125,3454,JP-R-381,668.0,-287.0,None,NaN,0,"(Id="""")",None,None
2126,3455,JP-X-348,648.0,-300.0,None,NaN,0,"(Id="""")",None,None
2127,3456,JP-K-315,635.0,-320.0,None,NaN,0,"(Id="""")",None,None
2128,3457,JP-H-664,658.0,6.0,None,NaN,0,"(Id="""")",None,None
2129,3458,JP-C-719,679.0,40.0,None,NaN,0,"(Id="""")",None,None
2130,3459,JP-V-707,646.0,61.0,None,NaN,0,"(Id="""")",None,None


,StarSystemName,PosX,PosY,primarySlot,planet,SpectralType,Luminosity,SubType,Name,Cluster,ClusterOverlay,ClusterConstellation
0,A Place,-123.764,272.182,2,"[{'name': 'Sullivan', 'type': 'Giant Terrestri...",G,V,5,1511.0,"(Id=""MWFactionAsset:StoryCluster3"")",/Game/Campaign/CampaignArcs/Regions/StoryClust...,None
1,Aalzorg,583.881,256.931,2,"[{'name': 'Rowan', 'type': 'Dwarf Terrestrial'...",G,V,9,NaN,NaN,NaN,NaN
2,Abadan,-70.319,-95.949,5,"[{'name': 'Africus', 'type': 'Dwarf Terrestria...",G,V,2,476.0,"(Id="""")",None,None
3,Abagnar,380.231,314.823,3,"[{'name': 'Bin Amr', 'type': 'Terrestrial', 'o...",G,V,9,2048.0,"(Id="""")",None,None
4,Abbadiyah,-317.967,246.551,2,"[{'name': 'Aswad', 'type': 'Ice Giant', 'orbit...",G,V,2,1337.0,"(Id="""")",None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
3738,TFS 38,119,1584,NaN,"[{'name': 'H1', 'type': 'Terrestrial', 'orbita...",G,V,9,NaN,NaN,NaN,NaN
3739,TFS 39,113,1612,NaN,"[{'name': 'MA.1', 'type': 'Giant Terrestrial',...",G,V,5,NaN,NaN,NaN,NaN
3740,TFS 40,107,1640,NaN,"[{'name': 'CR1', 'type': 'Terrestrial', 'orbit...",G,V,4,NaN,NaN,NaN,NaN
3741,TFS 41,101,1668,NaN,"[{'name': '8V9.1', 'type': 'Giant Terrestrial'...",M,V,2,NaN,NaN,NaN,NaN


,StarSystemName,PosX,PosY,primarySlot,planet,SpectralType,Luminosity,SubType,Name,Cluster,ClusterOverlay,ClusterConstellation
0,A Place,-123.764,272.182,2,"[{'name': 'Sullivan', 'type': 'Giant Terrestri...",G,V,5,1511.0,"(Id=""MWFactionAsset:StoryCluster3"")",/Game/Campaign/CampaignArcs/Regions/StoryClust...,None
1,Aalzorg,583.881,256.931,2,"[{'name': 'Rowan', 'type': 'Dwarf Terrestrial'...",G,V,9,NaN,NaN,NaN,NaN
2,Abadan,-70.319,-95.949,5,"[{'name': 'Africus', 'type': 'Dwarf Terrestria...",G,V,2,476.0,"(Id="""")",None,None
3,Abagnar,380.231,314.823,3,"[{'name': 'Bin Amr', 'type': 'Terrestrial', 'o...",G,V,9,2048.0,"(Id="""")",None,None
4,Abbadiyah,-317.967,246.551,2,"[{'name': 'Aswad', 'type': 'Ice Giant', 'orbit...",G,V,2,1337.0,"(Id="""")",None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
2168,HD-389202,370.0,485.0,NaN,NaN,None,NaN,0,3497.0,"(Id="""")",None,None
2169,HD-348126,370.0,510.0,NaN,NaN,None,NaN,0,3498.0,"(Id="""")",None,None
2170,HD-367482,350.0,530.0,NaN,NaN,None,NaN,0,3499.0,"(Id="""")",None,None
2171,HD-438297,345.0,552.0,NaN,NaN,None,NaN,0,3500.0,"(Id="""")",None,None


In [7]:
# parse faction changes from system_events.xml
with open(r'F:\system_events.xml', encoding='utf8') as file:
    tree = et.parse(file)
    root = tree.getroot()

    df = processed_systems.set_index('StarSystemName') # dataframe for primary planet lookup
    #display(df)
    #display(processed_systems.loc[processed_systems['id'] == 'Terra'])
    #row = processed_systems.loc[processed_systems['id'] == 'Terra']
    #display(row)
    #display(row.at[2754, 'primarySlot'])
    primarySlot = 0
    #display(primarySlot)

    # Desired fields
    system_name = []
    primary = []
    faction_change = [] # should be ((Date="",Faction=""), (Date="",Faction=""))

    dict = {'StarSystemName': system_name, 'primarySlot': primary, 'FactionChange': faction_change}

    for system in root.iter('system'):
        for id in system.iter('id'):
            system_name.append(id.text)
            # lookup primary planet since we can't account for individual planet ownership
            primarySlot = df.loc[id.text]['primarySlot']
            #print(primarySlot)
            #primarySlot = processed_systems.loc[processed_systems['id'] == ]
            #print(primarySlot)
        for planet in system.findall(f'.//planet/sysPos/[.="{primarySlot}"]/..'):
            primary.append(primarySlot)
            change = []
            for event in system.findall('.//event/faction/..'):
                date = event.find('date')
                faction = event.find('faction')
                change.append({'Date': date.text, 'Faction': faction.text})
            faction_change.append(change)
    # Create dataframe
    faction_changes = pd.DataFrame(dict)
    faction_changes.drop('primarySlot', axis=1, inplace=True)
    display(faction_changes)

processed_systems = pd.merge(processed_systems, faction_changes, how='left', on='StarSystemName') # merge into master data frame
display(processed_systems)
#changes.to_json(r'F:\events.json', orient='records', indent=2)

,StarSystemName,FactionChange
0,A Place,"[{'Date': '2235-04-30', 'Faction': 'IND'}, {'D..."
1,Aalzorg,"[{'Date': '2653-09-06', 'Faction': 'OA'}, {'Da..."
2,Abadan,"[{'Date': '2168-01-31', 'Faction': 'TA'}, {'Da..."
3,Abagnar,"[{'Date': '2712-06-25', 'Faction': 'DC'}]"
4,Abbadiyah,"[{'Date': '2423-01-19', 'Faction': 'LA'}, {'Da..."
...,...,...
3141,Transfer Station P9,"[{'Date': '3052-03-09', 'Faction': 'CW'}, {'Da..."
3142,Trelleborg,"[{'Date': '3049-08-17', 'Faction': 'CGB'}, {'D..."
3143,Trinity,"[{'Date': '3052-07-29', 'Faction': 'CJF'}, {'D..."
3144,Wark,"[{'Date': '3049-04-21', 'Faction': 'CJF'}, {'D..."


,StarSystemName,PosX,PosY,primarySlot,planet,SpectralType,Luminosity,SubType,Name,Cluster,ClusterOverlay,ClusterConstellation,FactionChange
0,A Place,-123.764,272.182,2,"[{'name': 'Sullivan', 'type': 'Giant Terrestri...",G,V,5,1511.0,"(Id=""MWFactionAsset:StoryCluster3"")",/Game/Campaign/CampaignArcs/Regions/StoryClust...,None,"[{'Date': '2235-04-30', 'Faction': 'IND'}, {'D..."
1,Aalzorg,583.881,256.931,2,"[{'name': 'Rowan', 'type': 'Dwarf Terrestrial'...",G,V,9,NaN,NaN,NaN,NaN,"[{'Date': '2653-09-06', 'Faction': 'OA'}, {'Da..."
2,Abadan,-70.319,-95.949,5,"[{'name': 'Africus', 'type': 'Dwarf Terrestria...",G,V,2,476.0,"(Id="""")",None,None,"[{'Date': '2168-01-31', 'Faction': 'TA'}, {'Da..."
3,Abagnar,380.231,314.823,3,"[{'name': 'Bin Amr', 'type': 'Terrestrial', 'o...",G,V,9,2048.0,"(Id="""")",None,None,"[{'Date': '2712-06-25', 'Faction': 'DC'}]"
4,Abbadiyah,-317.967,246.551,2,"[{'name': 'Aswad', 'type': 'Ice Giant', 'orbit...",G,V,2,1337.0,"(Id="""")",None,None,"[{'Date': '2423-01-19', 'Faction': 'LA'}, {'Da..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3791,HD-389202,370.0,485.0,NaN,NaN,None,NaN,0,3497.0,"(Id="""")",None,None,NaN
3792,HD-348126,370.0,510.0,NaN,NaN,None,NaN,0,3498.0,"(Id="""")",None,None,NaN
3793,HD-367482,350.0,530.0,NaN,NaN,None,NaN,0,3499.0,"(Id="""")",None,None,NaN
3794,HD-438297,345.0,552.0,NaN,NaN,None,NaN,0,3500.0,"(Id="""")",None,None,NaN


In [8]:
#processed_systems.loc[processed_systems['StarSystemName'].str.contains('HD-')]

#parse vanilla faction changes for HD- systems
vanilla_changes = pd.read_csv(r'F:\SystemFactionChanges.csv', encoding='utf16')
vanilla_changes = vanilla_changes.iloc[3151:]
vanilla_changes.drop(['---', 'Primary'], axis=1, inplace=True)
vanilla_changes.rename(columns = {'Name':'StarSystemName'}, inplace=True)
display(vanilla_changes)

# merge changes
processed_systems = pd.merge(processed_systems, vanilla_changes, how='left', on='StarSystemName')
display(processed_systems)
processed_systems['FactionChange'] = processed_systems['FactionChange_x'].combine_first(processed_systems['FactionChange_y'])
processed_systems.drop(['FactionChange_x', 'FactionChange_y'], axis=1, inplace=True)
display(processed_systems)

,StarSystemName,FactionChange
3151,HD-276914,"((Date=""2900-01-01"",Faction=""PIND""))"
3152,HD-417328,"((Date=""2900-01-01"",Faction=""PIND""))"
3153,HD-389202,"((Date=""2900-01-01"",Faction=""PIND""))"
3154,HD-348126,"((Date=""2900-01-01"",Faction=""PIND""))"
3155,HD-367482,"((Date=""2900-01-01"",Faction=""PIND""))"
3156,HD-438297,"((Date=""2900-01-01"",Faction=""PIND""))"
3157,HD-384971,"((Date=""2900-01-01"",Faction=""PIND""))"


,StarSystemName,PosX,PosY,primarySlot,planet,SpectralType,Luminosity,SubType,Name,Cluster,ClusterOverlay,ClusterConstellation,FactionChange_x,FactionChange_y
0,A Place,-123.764,272.182,2,"[{'name': 'Sullivan', 'type': 'Giant Terrestri...",G,V,5,1511.0,"(Id=""MWFactionAsset:StoryCluster3"")",/Game/Campaign/CampaignArcs/Regions/StoryClust...,None,"[{'Date': '2235-04-30', 'Faction': 'IND'}, {'D...",NaN
1,Aalzorg,583.881,256.931,2,"[{'name': 'Rowan', 'type': 'Dwarf Terrestrial'...",G,V,9,NaN,NaN,NaN,NaN,"[{'Date': '2653-09-06', 'Faction': 'OA'}, {'Da...",NaN
2,Abadan,-70.319,-95.949,5,"[{'name': 'Africus', 'type': 'Dwarf Terrestria...",G,V,2,476.0,"(Id="""")",None,None,"[{'Date': '2168-01-31', 'Faction': 'TA'}, {'Da...",NaN
3,Abagnar,380.231,314.823,3,"[{'name': 'Bin Amr', 'type': 'Terrestrial', 'o...",G,V,9,2048.0,"(Id="""")",None,None,"[{'Date': '2712-06-25', 'Faction': 'DC'}]",NaN
4,Abbadiyah,-317.967,246.551,2,"[{'name': 'Aswad', 'type': 'Ice Giant', 'orbit...",G,V,2,1337.0,"(Id="""")",None,None,"[{'Date': '2423-01-19', 'Faction': 'LA'}, {'Da...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3791,HD-389202,370.0,485.0,NaN,NaN,None,NaN,0,3497.0,"(Id="""")",None,None,NaN,"((Date=""2900-01-01"",Faction=""PIND""))"
3792,HD-348126,370.0,510.0,NaN,NaN,None,NaN,0,3498.0,"(Id="""")",None,None,NaN,"((Date=""2900-01-01"",Faction=""PIND""))"
3793,HD-367482,350.0,530.0,NaN,NaN,None,NaN,0,3499.0,"(Id="""")",None,None,NaN,"((Date=""2900-01-01"",Faction=""PIND""))"
3794,HD-438297,345.0,552.0,NaN,NaN,None,NaN,0,3500.0,"(Id="""")",None,None,NaN,"((Date=""2900-01-01"",Faction=""PIND""))"


,StarSystemName,PosX,PosY,primarySlot,planet,SpectralType,Luminosity,SubType,Name,Cluster,ClusterOverlay,ClusterConstellation,FactionChange
0,A Place,-123.764,272.182,2,"[{'name': 'Sullivan', 'type': 'Giant Terrestri...",G,V,5,1511.0,"(Id=""MWFactionAsset:StoryCluster3"")",/Game/Campaign/CampaignArcs/Regions/StoryClust...,None,"[{'Date': '2235-04-30', 'Faction': 'IND'}, {'D..."
1,Aalzorg,583.881,256.931,2,"[{'name': 'Rowan', 'type': 'Dwarf Terrestrial'...",G,V,9,NaN,NaN,NaN,NaN,"[{'Date': '2653-09-06', 'Faction': 'OA'}, {'Da..."
2,Abadan,-70.319,-95.949,5,"[{'name': 'Africus', 'type': 'Dwarf Terrestria...",G,V,2,476.0,"(Id="""")",None,None,"[{'Date': '2168-01-31', 'Faction': 'TA'}, {'Da..."
3,Abagnar,380.231,314.823,3,"[{'name': 'Bin Amr', 'type': 'Terrestrial', 'o...",G,V,9,2048.0,"(Id="""")",None,None,"[{'Date': '2712-06-25', 'Faction': 'DC'}]"
4,Abbadiyah,-317.967,246.551,2,"[{'name': 'Aswad', 'type': 'Ice Giant', 'orbit...",G,V,2,1337.0,"(Id="""")",None,None,"[{'Date': '2423-01-19', 'Faction': 'LA'}, {'Da..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3791,HD-389202,370.0,485.0,NaN,NaN,None,NaN,0,3497.0,"(Id="""")",None,None,"((Date=""2900-01-01"",Faction=""PIND""))"
3792,HD-348126,370.0,510.0,NaN,NaN,None,NaN,0,3498.0,"(Id="""")",None,None,"((Date=""2900-01-01"",Faction=""PIND""))"
3793,HD-367482,350.0,530.0,NaN,NaN,None,NaN,0,3499.0,"(Id="""")",None,None,"((Date=""2900-01-01"",Faction=""PIND""))"
3794,HD-438297,345.0,552.0,NaN,NaN,None,NaN,0,3500.0,"(Id="""")",None,None,"((Date=""2900-01-01"",Faction=""PIND""))"


In [9]:
# parse system_namechanges.xml
with open(r'F:\system_namechanges.xml', encoding='utf8') as file:
    tree = et.parse(file)
    root = tree.getroot()

    df = processed_systems.set_index('StarSystemName') # dataframe for primary planet lookup
    #display(df)
    #primarySlot = df[df['StarSystemName'] == 'Terra Firma']['primarySlot']
    primarySlot = 0

    # Desired fields
    system_name = []
    primary = []
    name_change = []

    dict = {'StarSystemName': system_name, 'primarySlot': primary, 'NameChange': name_change}

    for system in root.iter('system'):
        for id in system.iter('id'):
            #print(type(system_name))
            system_name.append(id.text)
            # lookup primary planet since we can't account for individual planet ownership
            primarySlot = df.loc[id.text]['primarySlot']
            #print(primarySlot)
            #primarySlot = processed_systems.loc[processed_systems['id'] == ]
            #print(primarySlot)
        for planet in system.findall(f'.//planet/sysPos/[.="{primarySlot}"]/..'):
            primary.append(primarySlot)
            change = []
            for event in system.findall('.//event/name/..'):
                date = event.find('date')
                name = event.find('name')
                change.append({'Date': date.text, 'Name': name.text})
            name_change.append(change)
    # Create dataframe
    name_changes = pd.DataFrame(dict)
    name_changes.drop('primarySlot', axis=1, inplace=True)
    display(name_changes)
    
processed_systems = pd.merge(processed_systems, name_changes, how='left', on='StarSystemName') # merge into master data frame
display(processed_systems)

,StarSystemName,NameChange
0,Adhara (Trip 3040+),"[{'Date': '0000-01-01', 'Name': 'Adhara'}, {'D..."
1,Aer (Finnalon 2822+),"[{'Date': '0000-01-01', 'Name': 'Aer'}, {'Date..."
2,Agmond (Bornal 3130+),"[{'Date': '0000-01-01', 'Name': 'Agmond'}, {'D..."
3,Anatolia (Pillory 3130+),"[{'Date': '0000-01-01', 'Name': 'Anatolia'}, {..."
4,Andiron (Helbrent 3025-),"[{'Date': '0000-01-01', 'Name': 'Helbrent'}, {..."
5,Arn (Jia Tian 3130+),"[{'Date': '0000-01-01', 'Name': 'Arn'}, {'Date..."
6,Austerlitz (Scauld 3130+),"[{'Date': '0000-01-01', 'Name': 'Austerlitz'},..."
7,Baltazar III (Himmels 3025-),"[{'Date': '0000-01-01', 'Name': 'Helbrent'}, {..."
8,Barlow's Folly (Cussar 2864-),"[{'Date': '0000-01-01', 'Name': 'Cussar'}, {'D..."
9,Belamor (Zanzor 2822+),"[{'Date': '0000-01-01', 'Name': 'Belamor'}, {'..."


,StarSystemName,PosX,PosY,primarySlot,planet,SpectralType,Luminosity,SubType,Name,Cluster,ClusterOverlay,ClusterConstellation,FactionChange,NameChange
0,A Place,-123.764,272.182,2,"[{'name': 'Sullivan', 'type': 'Giant Terrestri...",G,V,5,1511.0,"(Id=""MWFactionAsset:StoryCluster3"")",/Game/Campaign/CampaignArcs/Regions/StoryClust...,None,"[{'Date': '2235-04-30', 'Faction': 'IND'}, {'D...",NaN
1,Aalzorg,583.881,256.931,2,"[{'name': 'Rowan', 'type': 'Dwarf Terrestrial'...",G,V,9,NaN,NaN,NaN,NaN,"[{'Date': '2653-09-06', 'Faction': 'OA'}, {'Da...",NaN
2,Abadan,-70.319,-95.949,5,"[{'name': 'Africus', 'type': 'Dwarf Terrestria...",G,V,2,476.0,"(Id="""")",None,None,"[{'Date': '2168-01-31', 'Faction': 'TA'}, {'Da...",NaN
3,Abagnar,380.231,314.823,3,"[{'name': 'Bin Amr', 'type': 'Terrestrial', 'o...",G,V,9,2048.0,"(Id="""")",None,None,"[{'Date': '2712-06-25', 'Faction': 'DC'}]",NaN
4,Abbadiyah,-317.967,246.551,2,"[{'name': 'Aswad', 'type': 'Ice Giant', 'orbit...",G,V,2,1337.0,"(Id="""")",None,None,"[{'Date': '2423-01-19', 'Faction': 'LA'}, {'Da...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3791,HD-389202,370.0,485.0,NaN,NaN,None,NaN,0,3497.0,"(Id="""")",None,None,"((Date=""2900-01-01"",Faction=""PIND""))",NaN
3792,HD-348126,370.0,510.0,NaN,NaN,None,NaN,0,3498.0,"(Id="""")",None,None,"((Date=""2900-01-01"",Faction=""PIND""))",NaN
3793,HD-367482,350.0,530.0,NaN,NaN,None,NaN,0,3499.0,"(Id="""")",None,None,"((Date=""2900-01-01"",Faction=""PIND""))",NaN
3794,HD-438297,345.0,552.0,NaN,NaN,None,NaN,0,3500.0,"(Id="""")",None,None,"((Date=""2900-01-01"",Faction=""PIND""))",NaN


In [10]:
processed_systems.sort_values(by='Name', inplace=True, ignore_index=True, na_position='first')
filler = pd.Series(range(10000, 20000, 1))
processed_systems.Name.fillna(filler, inplace=True)
display(processed_systems)
processed_systems.sort_values(by='Name', inplace=True, ignore_index=True, na_position='first')
#print(processed_systems["Name"].is_unique)

,StarSystemName,PosX,PosY,primarySlot,planet,SpectralType,Luminosity,SubType,Name,Cluster,ClusterOverlay,ClusterConstellation,FactionChange,NameChange
0,Aalzorg,583.881,256.931,2,"[{'name': 'Rowan', 'type': 'Dwarf Terrestrial'...",G,V,9,10000.0,NaN,NaN,NaN,"[{'Date': '2653-09-06', 'Faction': 'OA'}, {'Da...",NaN
1,Able's Glory,-539.23,124.7,4,"[{'name': 'Garbrecht', 'type': 'Dwarf Terrestr...",F,V,5,10001.0,NaN,NaN,NaN,"[{'Date': '3130-12-03', 'Faction': 'RIM'}]",NaN
2,Aboukir,-419.964,287.637,1,"[{'name': 'Aboukir', 'type': 'Terrestrial', 'o...",K,V,6,10002.0,NaN,NaN,NaN,"[{'Date': '2663-12-03', 'Faction': 'RWR'}, {'D...",NaN
3,Abushiri,-267.585,326.475,2,"[{'name': 'Priest River', 'type': 'Ice Giant',...",K,V,2,10003.0,NaN,NaN,NaN,"[{'Date': '2660-03-28', 'Faction': 'RWR'}, {'D...",NaN
4,Achilles,44.858,-132.111,2,"[{'name': 'Odur', 'type': 'Dwarf Terrestrial',...",K,V,2,10004.0,NaN,NaN,NaN,"[{'Date': '2659-11-25', 'Faction': 'CC'}, {'Da...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3791,HD-389202,370.0,485.0,NaN,NaN,None,NaN,0,3497.0,"(Id="""")",None,None,"((Date=""2900-01-01"",Faction=""PIND""))",NaN
3792,HD-348126,370.0,510.0,NaN,NaN,None,NaN,0,3498.0,"(Id="""")",None,None,"((Date=""2900-01-01"",Faction=""PIND""))",NaN
3793,HD-367482,350.0,530.0,NaN,NaN,None,NaN,0,3499.0,"(Id="""")",None,None,"((Date=""2900-01-01"",Faction=""PIND""))",NaN
3794,HD-438297,345.0,552.0,NaN,NaN,None,NaN,0,3500.0,"(Id="""")",None,None,"((Date=""2900-01-01"",Faction=""PIND""))",NaN


In [11]:
# save new inner sphere data file
inner_sphere_data = processed_systems[['Name', 'StarSystemName', 'PosX', 'PosY', 'SpectralType', 'Luminosity', 'SubType',
                                      'Cluster', 'ClusterOverlay', 'ClusterConstellation']]
display(inner_sphere_data)
#inner_sphere_data.columns = ['Name', 'StarSystemName', 'PosX', 'PosY', 'SpectralType', 'Luminosity', 'SubType',
#        'Cluster', 'ClusterOverlay', 'ClusterConstellation']
display(inner_sphere_data)
inner_sphere_data.to_json(r'F:\inner_sphere_data.json', indent=2, orient='records')

,Name,StarSystemName,PosX,PosY,SpectralType,Luminosity,SubType,Cluster,ClusterOverlay,ClusterConstellation
0,0.0,INVALID,0.0,0.0,None,NaN,0,"(Id="""")",None,None
1,1.0,Terra,0,0,G,V,2,"(Id=""MWFactionAsset:RepairSystem_26"")",None,None
2,2.0,New Earth (Tau Ceti 2116-),-12.415,3.055,F,V,0,"(Id=""MWFactionAsset:RepairSystem_26"")",/Game/Campaign/CampaignArcs/BorderChanges/_com...,/Game/Campaign/CampaignArcs/BorderChanges/_com...
3,3.0,Rigil Kentarus,-2.852,2.54,G,V,8,"(Id=""MWFactionAsset:RepairSystem_26"")",None,None
4,4.0,Pike IV,20.153,39.24,K,IV,9,"(Id="""")",None,None
...,...,...,...,...,...,...,...,...,...,...
3791,11678.0,TFS 38,119,1584,G,V,9,NaN,NaN,NaN
3792,11679.0,TFS 39,113,1612,G,V,5,NaN,NaN,NaN
3793,11680.0,TFS 40,107,1640,G,V,4,NaN,NaN,NaN
3794,11681.0,TFS 41,101,1668,M,V,2,NaN,NaN,NaN


,Name,StarSystemName,PosX,PosY,SpectralType,Luminosity,SubType,Cluster,ClusterOverlay,ClusterConstellation
0,0.0,INVALID,0.0,0.0,None,NaN,0,"(Id="""")",None,None
1,1.0,Terra,0,0,G,V,2,"(Id=""MWFactionAsset:RepairSystem_26"")",None,None
2,2.0,New Earth (Tau Ceti 2116-),-12.415,3.055,F,V,0,"(Id=""MWFactionAsset:RepairSystem_26"")",/Game/Campaign/CampaignArcs/BorderChanges/_com...,/Game/Campaign/CampaignArcs/BorderChanges/_com...
3,3.0,Rigil Kentarus,-2.852,2.54,G,V,8,"(Id=""MWFactionAsset:RepairSystem_26"")",None,None
4,4.0,Pike IV,20.153,39.24,K,IV,9,"(Id="""")",None,None
...,...,...,...,...,...,...,...,...,...,...
3791,11678.0,TFS 38,119,1584,G,V,9,NaN,NaN,NaN
3792,11679.0,TFS 39,113,1612,G,V,5,NaN,NaN,NaN
3793,11680.0,TFS 40,107,1640,G,V,4,NaN,NaN,NaN
3794,11681.0,TFS 41,101,1668,M,V,2,NaN,NaN,NaN


In [12]:
# save faction changes
faction_changes = processed_systems[['Name', 'StarSystemName', 'FactionChange']]
display(faction_changes)
faction_changes.to_json(r'F:\faction_changes.json', indent=2, orient='records')

,Name,StarSystemName,FactionChange
0,0.0,INVALID,NaN
1,1.0,Terra,"[{'Date': '2205-04-16', 'Faction': 'TA'}, {'Da..."
2,2.0,New Earth (Tau Ceti 2116-),"[{'Date': '2116-04-04', 'Faction': 'TA'}, {'Da..."
3,3.0,Rigil Kentarus,"[{'Date': '2270-10-16', 'Faction': 'TA'}, {'Da..."
4,4.0,Pike IV,"[{'Date': '2242-12-25', 'Faction': 'IND'}, {'D..."
...,...,...,...
3791,11678.0,TFS 38,NaN
3792,11679.0,TFS 39,NaN
3793,11680.0,TFS 40,NaN
3794,11681.0,TFS 41,NaN


In [13]:
# save name changes
name_changes = processed_systems[['Name', 'StarSystemName', 'NameChange']]
display(name_changes)
name_changes.to_json(r'F:\name_changes.json', indent=2, orient='records')

,Name,StarSystemName,NameChange
0,0.0,INVALID,NaN
1,1.0,Terra,NaN
2,2.0,New Earth (Tau Ceti 2116-),"[{'Date': '0000-01-01', 'Name': 'Tau Ceti'}, {..."
3,3.0,Rigil Kentarus,NaN
4,4.0,Pike IV,NaN
...,...,...,...
3791,11678.0,TFS 38,NaN
3792,11679.0,TFS 39,NaN
3793,11680.0,TFS 40,NaN
3794,11681.0,TFS 41,NaN
